In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask import grid

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

img_paths, label_paths = wflow.grid_images()

wflow.remove_from_gridded(img_paths, label_paths)

wflow.move_chips_to_folder()

In [ ]:
wflow.connected_components()

In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask import grid

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

In [ ]:
%%time
with rasterio.open(in_path) as inds:
    chip_list = get_tiles_for_threaded_map(inds)

In [ ]:
%load_ext autoreload
%autoreload 2
from cropmask import grid

In [ ]:
wflow.stacked_path

In [ ]:
wflow.GRIDDED_IMGS

In [ ]:
%%time
out_paths = grid.grid_images_rasterio_controlled_threads('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/mnt/point/landsat-1024-cp/landsat_grid_imgs')

In [ ]:
%%time
out_paths = grid_images_rasterio_regular_map(wflow.stacked_path, wflow.GRIDDED_IMGS)